In [1]:
import numpy as np

LIMIT = 10
class Dataset:
    def __init__(self, LIMIT=10, dir_path="drive/MyDrive/Datasets"):
        self.dataset = [[[] for _ in range(10)] for _ in range(10)]
        self.data = []
        self.labels = []
        limit_the_search = [j for i in range(10) for k in range(10) for j in range(i*100+(k*10)+1, i*100+(k*10)+LIMIT+1)]
        for i in limit_the_search:
          filepath = "{}/{}.txt".format(dir_path, i)
          try:
              lines = [line.strip() for line in open(filepath, "r")]
              class_id =  int(lines[1][len(lines[1]) - 1])
              user_id = int(lines[2][len(lines[2]) - 1])
              user_data = []
              for i, line in enumerate(lines[5:]):
                  if line:
                      user_data.append(list(map(float, line.split(',')))[0:3])
              self.dataset[user_id - 1][class_id - 1].append(user_data) 
              self.data.append(user_data)
              self.labels.append(class_id)
          except IOError as e:
              print("Unable to read dataset file {}!\n".format(filepath))

    def get_user(self, index):
        """Returns an array containing all the data saved for the user <<index>>"""
        return len(self.data[index])


In [2]:
data = Dataset()

# K-Nearest Neighbors & DTW

In [3]:
import numpy as np
from sklearn.metrics import accuracy_score
from scipy.stats import mode
from joblib import Parallel, delayed
from numba import njit, prange
from numba.typed import List

@njit()
def distance(x, y):
  dist = 0.
  for di in range(len(x)):
      diff = (x[di] - y[di])
      dist += diff * diff
  return dist

@njit(nogil=True)
def dtw_distance(time_serie1, time_serie2):
  l1, l2 = len(time_serie1), len(time_serie2)
  cost_matrix = np.full((l1 + 1, l2 + 1), np.inf)
  # Initialize the first cell
  cost_matrix[0, 0] = 0. 
  # populatte the cost matrix within window
  for i in range(l1):
    for j in range(l2):
      cost_matrix[i+1, j+1] = distance(List(time_serie1[i]), List(time_serie2[j])) 
      cost_matrix[i+1, j+1] += min(cost_matrix[i, j +1], cost_matrix[i+1, j], cost_matrix[i, j])
  return np.sqrt(cost_matrix[1:, 1:])[-1, -1]

class KNN_DTW:
  """
  This class is a classifier using DTW as distance measure between pairs of time series data
  """
  def __init__(self, n_neighbors=1, warping_window=50):
    self.n_neighbors = n_neighbors
    self.warping_window = warping_window

  def fit(self, x, labels):
    self.x = np.array(x)
    self.labels = np.array(labels)
  

  def predict(self, x):
    dist_matrix = np.zeros((x.shape[0], self.x.shape[0]))
    matrix = Parallel(n_jobs=-1, prefer="threads", verbose=0)(
            delayed(dtw_distance)(
                List(x[i]), List(self.x[j])
            )
            for i in range(len(x)) for j in range(len(self.x))
        )
    dist_matrix = np.array(matrix).reshape((len(x), -1))
    # the index of the k nearest neighbors
    indexes = dist_matrix.argsort()[:, :self.n_neighbors]
    # identifiers the neigbors
    labels = self.labels[indexes]
    # get the majority votes
    predictions = mode(labels, axis=1)[0]
    #print(predictions)
    return predictions

# Validation function
def test(user_id, dataset, labels, model, LIMIT=100):
  # split the dataset
  indexes = range(user_id*LIMIT, user_id*LIMIT+LIMIT+1)
  train_set = np.delete(dataset, indexes)
  train_labels = np.delete(labels, indexes)
  test_labels = labels[indexes]
  test_set = dataset[indexes]
  # Prediction
  model.fit(train_set, train_labels)
  predictions = model.predict(test_set)
  return accuracy_score(test_labels, predictions)


def validation(dataset, labels, model, LIMIT=100):
  dataset = np.array(dataset)
  labels = np.array(labels)
  accuracies = []
  for user_id in range(10):
    accuracies.append(test(user_id, dataset, labels, model, LIMIT))
    print("The user score {}: {}".format(user_id+1, accuracies[-1]))
  return accuracies


In [ ]:
model = KNN_DTW(1)
accuracies = validation(np.array(data.data).T, data.labels, model, 100)
accuracies

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/usr/local/lib/python3.7/dist-packages/numba/typed/typedlist.py:66: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'item' of function 'impl_append.<locals>.impl'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../usr/local/lib/python3.7/dist-packages/numba/typed/listobject.py", line 596:

    def impl(l, item):
    ^

  l.append(item)
/usr/local/lib/python3.7/dist-packages/numba/core/ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type t

# Using another library